# Genetic Algorithms example: the knapsack problem

Hi! 

In this notebook we'll work out through one of the most basic NP-complete problems - the Knapsack Problem:

https://en.wikipedia.org/wiki/Knapsack_problem

Basically, we want to pack items into the knapsack. Each **item** has its **value and weight** and the **knapsack** has some **finite capacity**. The target is to maximize the value of packed items while not exceeding the capacity of our knapsack.

Though this description is quite simple, the naive algorithm to find best solution for n items has a complexity of $O(2^n)$ _(why?)_.

This is _bad_.

We'll see how it can be improved with a genetic algorithm!

https://en.wikipedia.org/wiki/Genetic_algorithm

The knapsack problem is defined by:

* $c$ - capacity of knapsack
* $n$ - number of items
* $v_i$ - value of i-th item
* $w_i$ - weight of i-th item

We'll be using an example problem instance generated below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
class KnapsackInstance:
    def __init__(self, c, n, v, w):
        self.c = c
        self.n = n
        self.v = v
        self.w = w
        
    def __str__(self):
        return 'c = {}\nn = {}\nv = {}\nw = {}'.format(c, n, v, w)

In [ ]:
c = 100
n = 100
v = 10 * np.random.rand(n)
w = 10 * np.random.rand(n)

p = KnapsackInstance(c, n, v, w)

We have to represent our solution in terms of genetic algorithm. In our case, the chromosome will be an array of 0s and 1s, which indicate whether we pack a given item into the knapsack (1) or not (0) and initial population will be generated randomly.

In [ ]:
def weight(p, chromosome):
    return np.sum(np.multiply(p.w, chromosome))

To make sure our chromosomes don't exceed the capacity of the knapsack, we'll define a helper method which randomly throws out items until the capacity is not exceeded:

In [ ]:
def make_chromosome_great_again(p, chromosome):
    w = weight(p, chromosome)
    picked = (chromosome == 1).nonzero()[0] # indices of picked items
    while w > p.c:
        i = np.random.randint(picked.shape[0])
        unpick = picked[i]
        picked = np.delete(picked, i)
        chromosome[unpick] = 0
        w -= p.w[unpick]
    
    return chromosome

The chromosomes generator:

In [ ]:
def make_random_chromosome(p):
    c = np.array([random.randint(0,1) for _ in range(p.n)])
    return make_chromosome_great_again(p, c)

In [ ]:
def make_random_population(p, size):
    return np.array([make_random_chromosome(p) for _ in range(size)])

And a helper function to get the best chromosome of the population:

In [ ]:
def best(p, population):
    return max(population, key=lambda c: fitness(p, c))

Next, we can define the fitness function - simply the value of the items that ended up in the knapsack:

In [ ]:
def fitness(p, chromosome):
    return np.sum(np.multiply(p.v, chromosome))

Next, we'll employ a roulette choice selection:

In [ ]:
def roulette_choice(p, population):
    fitness_sum = sum(fitness(p, c) for c in population)
    pick = np.random.rand() * fitness_sum
    current = 0
    for chromosome in population:
        current += fitness(p, chromosome)
        if current > pick:
            return chromosome

A simple crossover and breeding operation:

In [ ]:
def crossover(p, mother, father):
    split = np.random.randint(p.n)
    child = np.append(mother[:split], father[split:])
    return make_chromosome_great_again(p, child)

def breed(p, population):
    mother = roulette_choice(p, population)
    father = roulette_choice(p, population)
    return crossover(p, mother, father)

And a mutation:

In [ ]:
def mutation(p, population, mutation_probability=0.01):
    for i, c in enumerate(population):
        for j in range(p.n):
            if np.random.rand() < mutation_probability:
                c[j] = 1 - c[j] # swap 0 and 1
        population[i] = make_chromosome_great_again(p, c)
    return population

Now that we have all the parts, we can build a single evolution step for our population.

In [ ]:
def step(p, population, crossover_percentage=0.9, mutation_probability=0.001, keep_best=True):
    
    # first, we select the parents, breed new children and create a new population from them
    child_count = int(crossover_percentage * len(population))
    new_population = [breed(p, population) for _ in range(child_count)]
    
    # the children mutate
    new_population = mutation(p, new_population, mutation_probability=mutation_probability)

    if keep_best:
        # we also make sure the best of the original chromosomes ends up in the new population
        new_population.append(best(p, population))
    
    # finally, we select random parents to remain in the new population, so that it doesn't shrink
    parent_indices = np.random.choice(population.shape[0], len(population) - 1 - child_count, replace=False)
    parent_sample = population[parent_indices]
    new_population.extend(parent_sample)

    return np.array(new_population)

Finally, we can define some basic parameters for the algorithm and go!

In [ ]:
generations = 100
population_size = 100
population = make_random_population(p, 100)
crossover_percentage = 0.8
mutation_probability = 0.01
best_history = []

In [ ]:
for _ in range(generations):
    population = step(p, population, 
                      crossover_percentage=crossover_percentage,
                      mutation_probability=mutation_probability
                     )
    best_history.append(best(p, population))

x = np.arange(generations)
y = [fitness(p, b) for b in best_history]

plt.plot(x,y)
plt.show()